In [39]:
import os
import random
import pandas as pd
import geopandas as gpd
import plotly.express as px
import ipywidgets as widgets
# 加入字型
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Arial Unicode MS'

from plotnine import *
from plotly.graph_objs import FigureWidget
from IPython.display import display, clear_output

data = {}
data_name = {
    "犯罪總數(含駕駛過失)": "negligent_driving", 
    "查獲槍彈刀械": "guns_bullets_knives",
    "毒品": "poison", 
    "暴力犯罪": "violent_crime", 
    "竊盜": "theft"\
}
excels = [
    "犯罪總數(含駕駛過失).xlsx", "查獲槍彈刀械.xlsx", "毒品.xlsx",
    "暴力犯罪.xlsx", "竊盜.xlsx"
]

taiwan_citys = ["台灣全部","基隆市", "臺北市", "新北市", "桃園市", "苗栗縣", "新竹市", "新竹縣", "臺中市", 
                "彰化縣", "南投縣", "雲林縣", "嘉義市", "嘉義縣", "臺南市", "高雄市", "屏東縣",
                "臺東縣", "花蓮縣", "宜蘭縣", "澎湖縣", "金門縣", "連江縣"]

six_capitals_area = {
    "臺北市":["中正區", "大同區", "中山區", "松山區", "大安區", "萬華區", "信義區", "士林區", "北投區", "內湖區", "南港區", "文山區"],
    "新北市":["萬里區", "金山區", "板橋區", "汐止區", "深坑區", "石碇區", "瑞芳區", "平溪區", "雙溪區", "貢寮區", "新店區", "坪林區", "烏來區", "永和區", "中和區", "土城區", "三峽區", "樹林區", "鶯歌區", "三重區", "新莊區", "泰山區", "林口區", "蘆洲區", "五股區", "八里區", "淡水區", "三芝區", "石門區"],
    "桃園市":["中壢區", "平鎮區", "龍潭區", "楊梅區", "新屋區", "觀音區", "桃園區", "龜山區", "八德區", "大溪區", "復興區", "大園區", "蘆竹區"],
    "臺中市":["中區", "東區", "南區", "西區", "北區", "北屯區", "西屯區", "南屯區", "太平區", "大里區", "霧峰區", "烏日區", "豐原區", "后里區", "石岡區", "東勢區", "和平區", "新社區", "潭子區", "大雅區", "神岡區", "大肚區", "沙鹿區", "龍井區", "梧棲區", "清水區", "大甲區", "外埔區", "大安區"],
    "臺南市":["中西區", "東區", "南區", "北區", "安平區", "安南區", "永康區", "歸仁區", "新化區", "左鎮區", "玉井區", "楠西區", "南化區", "仁德區", "關廟區", "龍崎區", "官田區", "麻豆區", "佳里區", "西港區", "七股區", "將軍區", "學甲區", "北門區", "新營區", "後壁區", "白河區", "東山區", "六甲區", "下營區", "柳營區", "鹽水區", "善化區", "大內區", "山上區", "新市區", "安定區"],
    "高雄市":["新興區", "前金區", "苓雅區", "鹽埕區", "鼓山區", "旗津區", "前鎮區", "三民區", "楠梓區", "小港區", "左營區", "仁武區", "大社區", "岡山區", "路竹區", "阿蓮區", "田寮區", "燕巢區", "橋頭區", "梓官區", "彌陀區", "永安區", "湖內區", "鳳山區", "大寮區", "林園區", "鳥松區", "大樹區", "旗山區", "美濃區", "六龜區", "內門區", "杉林區", "甲仙區", "桃源區", "那瑪夏區", "茂林區", "茄萣區"]
}

# 將Data資料夾內的excel讀取出來, 存在data變數中
for excel in excels:
    excel_name = excel.split(".")[0]
    data[data_name[excel_name]] = pd.read_excel("./Data/{}".format(excel))
    data[data_name[excel_name]] = data[data_name[excel_name]].set_index('地區')
    data[data_name[excel_name]].index = data[data_name[excel_name]].index.str.strip()

output = widgets.Output()

# 第一頁
def homepage():
    # 需要顯示時, 才會出現, 避免畫面沒更新顯示
    with output:
        clear_output(wait=True)

        # 定義全局寬度
        common_layout = widgets.Layout(width='70%', margin="10px 0")

        # 製作第一頁：下拉式選單
        crime_class = widgets.Dropdown(
            options = ["全", "犯罪總數(含駕駛過失)", "查獲槍彈刀械", "毒品", "暴力犯罪", "竊盜"],
            description = "刑案類別 :",
            layout=common_layout
        )
        year = widgets.SelectMultiple(
            options = ["104年", "105年", "106年", "107年", "108年", "109年", "110年"],
            # value=['Option1'],
            description='年份 :',
            disabled=False,
            layout=common_layout
        )
        sort = widgets.Dropdown(
            options = ["遞增", "遞減", "不做排序"],
            description = "排序 :",
            layout=common_layout
        )
        city = widgets.Dropdown(
            options = ["台灣全部","基隆市", "臺北市", "新北市", "桃園市", "苗栗縣", "新竹市", "新竹縣", "臺中市", 
                       "彰化縣", "南投縣", "雲林縣", "嘉義市", "嘉義縣", "臺南市", "高雄市", "屏東縣",
                       "臺東縣", "花蓮縣", "宜蘭縣", "澎湖縣", "金門縣", "連江縣"],
            description = "縣市 :",
            layout=common_layout
        )
       
        # 搜尋按鈕
        search_btn = widgets.Button(description = "搜尋", tooltip = "搜尋", layout=widgets.Layout(width="70%", margin="37px 0 0 0"))
        search_btn.on_click(lambda b: resultpage(crime_class.label, year.value, sort.label, city.label))

        # 台灣地圖 圖片
        map_taiwan_file = open("./Data/image/map_taiwan.png", "rb")
        map_taiwan = map_taiwan_file.read()

        taiwan_img = widgets.Image(
            value=map_taiwan,
            format="png",
            layout=widgets.Layout(width="40%")
        )
        
        # 建立元件, 並以垂直的方式排列
        V = widgets.VBox([crime_class, year, sort, city, search_btn], layout=widgets.Layout(align_items="flex-start", width="60%"))
        H = widgets.HBox([V, taiwan_img], layout=widgets.Layout(align_items="center", justify_content="center", width="800px", height="600px")) 
        container = widgets.VBox([H], layout=widgets.Layout(align_items="center", justify_content="center", width="100%", height="100%"))

        # 顯示元件
        display(container)   

# 第二頁
def resultpage(crime_class, year, sort, city):
    if city == "台灣全部":
        fig, total = CreateFigure_taiwan(crime_class, year, sort, city)
    else:
        fig, total = CreateFigure(crime_class, year, sort, city)
    fig_widget = FigureWidget(fig)

    # 需要顯示時, 才會出現, 避免畫面沒更新顯示
    with output:
        clear_output(wait=True)
        # print(crime_class, year, sort, city)
        
        crime_class_label = widgets.Label(value=f"刑案類別 :  {crime_class}")
        year_text = " ".join(list(year))
        year_label = widgets.Label(value=f"年份 :  {year_text}")
        sort_label = widgets.Label(value=f"排序方式 :  {sort}", layout=widgets.Layout(width="200px"))
        city_label = widgets.Label(value=f"{city}地區")
        #city_label = widgets.Label(value=f"地點 :  {city}", layout=widgets.Layout(width="125px"))

        # 各縣市地圖
        map_city_file = open(f"./Data/image/map_{city}.png", "rb")  
        map_city = map_city_file.read()

        city_img = widgets.Image(
            value=map_city,
            format="png",
            layout=widgets.Layout(width="350px", height="300px")
        )
        
        # 顯示什麼地區資料, 台灣地圖 按鈕
        area_data = widgets.Button(description = f"{city}資料", tooltip = f"{city}資料", disabled = True, layout=widgets.Layout(width="50%"))

        # 設定只有六都會有地區深淺圖
        if city in ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市"]:
            map_taiwan_btn = widgets.Button(description = "區域", tooltip = "區域", layout=widgets.Layout(width="50%"))
            map_taiwan_btn.on_click(lambda b: hotpage(crime_class, year, sort, city))
        else:
            map_taiwan_btn = widgets.Button(description = "區域", tooltip = "區域", layout=widgets.Layout(width="50%"), disabled = True)

        # 回首頁按鈕
        home_btn = widgets.Button(description = "回首頁", tooltip = "回首頁", layout=widgets.Layout(width="350px"))
        home_btn.on_click(lambda b: homepage())

        total_label = widgets.Label(value=f"總人數 : {total:,}人")
        
        H1 = widgets.VBox([crime_class_label, year_label])
        H2 = widgets.HBox([city_label], layout=widgets.Layout(align_items="center", justify_content="center", font_size='18px'))
        H3 = widgets.HBox([area_data, map_taiwan_btn], layout=widgets.Layout(width="350px"))

        V1 = widgets.VBox([H1, sort_label], layout=widgets.Layout(align_items="flex-start", width="425px"))
        V2 = widgets.VBox([city_img, H3,home_btn, total_label], layout=widgets.Layout(align_items="flex-start", width="350px"))

        V = widgets.VBox([V1, H2, V2])
        H = widgets.HBox([V, fig_widget], layout=widgets.Layout(align_items="center", justify_content="center", width="1200px", height="600px"))
        container = widgets.VBox([H], layout=widgets.Layout(align_items="center", justify_content="center", width="100%", height="100%"))

        # 顯示元件
        display(container)

# 六都的第三頁
def hotpage(crime_class, year, sort, city):
    fig, total, df = CreateFigure_area(crime_class, year, sort, city)
    fig_widget = FigureWidget(fig)

    # 取得地區深淺圖, 轉乘Widgets.Image
    area_ggplot = Create_ggplot(city, df)
    area_ggplot.save('area_ggplot.png')
    plot_img = open('area_ggplot.png', 'rb').read()
    plot_widget = widgets.Image(value=plot_img, format='png', width=350, height=300)
    os.remove('area_ggplot.png')

    # 需要顯示時, 才會出現, 避免畫面沒更新顯示
    with output:
        clear_output(wait=True)
        # print(crime_class, year, sort, city)
        
        crime_class_label = widgets.Label(value=f"刑案類別 :  {crime_class}")
        year_text = " ".join(list(year))
        year_label = widgets.Label(value=f"年份 :  {year_text}")
        city_label = widgets.Label(value=f"{city}地區")
        #city_label = widgets.Label(value=f"地點 :  {city}", layout=widgets.Layout(width="125px"))

        # 各縣市地圖
        map_city_file = open(f"./Data/image/map_{city}.png", "rb")  
        map_city = map_city_file.read()
        
        # 顯示什麼地區資料, 台灣地圖 按鈕
        area_data = widgets.Button(description = f"{city}資料", tooltip = f"{city}資料", disabled = True, layout=widgets.Layout(width="50%"))
        map_taiwan_btn = widgets.Button(description = "區域", tooltip = "區域", layout=widgets.Layout(width="50%"), disabled = True)

        # 回首頁按鈕
        home_btn = widgets.Button(description = "回首頁", tooltip = "回首頁", layout=widgets.Layout(width="350px"))
        home_btn.on_click(lambda b: homepage())

        total_label = widgets.Label(value=f"總人數 : {total:,}人")
        
        H1 = widgets.VBox([crime_class_label, year_label])
        H2 = widgets.HBox([city_label], layout=widgets.Layout(align_items="center", justify_content="center", font_size='18px'))
        H3 = widgets.HBox([area_data, map_taiwan_btn], layout=widgets.Layout(width="350px"))

        #V1 = widgets.VBox([H1, sort_label], layout=widgets.Layout(align_items="flex-start", width="425px"))
        V2 = widgets.VBox([plot_widget, H3,home_btn, total_label], layout=widgets.Layout(align_items="flex-start", width="350px"))

        #V = widgets.VBox([V1, H2, V2])
        V = widgets.VBox([H1, H2, V2])
        H = widgets.HBox([V, fig_widget], layout=widgets.Layout(align_items="center", justify_content="center", width="1200px", height="600px"))
        container = widgets.VBox([H], layout=widgets.Layout(align_items="center", justify_content="center", width="100%", height="100%"))

        # 顯示元件
        display(container)

# 製作長條圖
def CreateFigure(crime_class, year, sort, city):
    result_data = {}

    if crime_class == "全":
        result_data = pd.Series(0, index=list(year))
        for k, v in data_name.items():
            result_data += data[v].loc[city, list(year)]
    else:
        result_data = data[data_name[crime_class]].loc[city, list(year)]
    #print(result_data)
    
    # 假設 result_data 是一個 pandas Series，我們可以轉換它為 DataFrame
    result_data = result_data.reset_index()
    result_data.columns = ['年份', '案件數量']
    #print(result_data)

    if sort == "遞增":
        result_data = result_data.sort_values(by=["案件數量"])
    elif sort == "遞減":
        result_data = result_data.sort_values(by=["案件數量"], ascending=False)

    # 創建長條圖
    figure = px.bar(result_data, x='年份', y='案件數量', title=f"{city}")
    figure.update_layout(title_x=0.5, height=600)
    figure.update_yaxes(tickformat=',')

    # 計算總人數
    total_people = result_data["案件數量"].sum()

    return figure, total_people

# 製作台灣長條圖
def CreateFigure_taiwan(crime_class, year, sort, city):
    if crime_class == "全":
        columns = ['104年', '105年', '106年', '107年', '108年', '109年', '110年']
        areas = ["新北市", "臺北市", "桃園市", "臺中市", "臺南市", "高雄市",
                    "宜蘭縣", "新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣",
                    "嘉義縣", "屏東縣", "臺東縣", "花蓮縣", "澎湖縣", "基隆市",
                    "新竹市", "嘉義市", "金門縣", "連江縣"]
        df_combined = pd.DataFrame(index=areas, columns=columns)
        df_combined.index.name = '地區'
        #print(df_combined)
        for k, v in data_name.items():
            df_combined = df_combined.add(data[v], fill_value=0)
        df = df_combined[list(year)].reset_index().melt(id_vars=['地區'], var_name='年份', value_name='案件數量')
    else:
        temp_data = data[data_name[crime_class]]
        df = temp_data[list(year)].reset_index().melt(id_vars=['地區'], var_name='年份', value_name='案件數量')
    
    total_cases_by_area = df.groupby('地區')['案件數量'].sum().reset_index()
    df = df.merge(total_cases_by_area, on='地區', suffixes=('', '_總數'))  
    if sort == "遞增":
        df = df.sort_values(by='案件數量_總數')
        #df = df.groupby('地區', group_keys=False).apply(lambda x: x.sort_values(['案件數量']))
        #df = df.sort_values(by=['地區', '案件數量'])
    elif sort == "遞減":
        df = df.sort_values(by='案件數量_總數', ascending=False)
        #df = df.groupby('地區', group_keys=False).apply(lambda x: x.sort_values(['案件數量'], ascending=False))
        #df = df.sort_values(by=['地區', '案件數量'], ascending=False)

    # 創建長條圖
    figure = px.bar(df, x='地區', y='案件數量', color='年份', barmode="group", title=f"{city}")
    figure.update_layout(title_x=0.5, height=600)
    figure.update_yaxes(tickformat=',')

    # 計算總人數
    total_people = df["案件數量"].sum()

    return figure, total_people

# 製作區域長條圖
def CreateFigure_area(crime_class, year, sort, city):
    result_data = {}
    area_data = {
        "地區":[],
        "年份":[],
        "案件數量":[]
    }

    if crime_class == "全":
        result_data = pd.Series(0, index=list(year))
        for k, v in data_name.items():
            result_data += data[v].loc[city, list(year)]
    else:
        result_data = data[data_name[crime_class]].loc[city, list(year)]
    #print(result_data)

    for year in result_data.keys():
        area_data["地區"].extend(six_capitals_area[city])
        area_data["年份"].extend([year for i in range(len(six_capitals_area[city]))])   

    for key, value in result_data.items():
        year_total = value
        nums = len(six_capitals_area[city])
        min_value_per_variable = 1
        list_of_cases = []

        assert year_total >= nums * min_value_per_variable
        list_of_cases = [min_value_per_variable] * nums
        remaining = year_total - nums * min_value_per_variable
        random.seed(0)
        for index in range(remaining):
            list_of_cases[random.randint(0, nums - 1)] +=1
        #print(list_of_cases)
        area_data["案件數量"].extend(list_of_cases)

    
    df = pd.DataFrame(area_data)
    total_cases_by_area = df.groupby('地區')['案件數量'].sum().reset_index()
    df = df.merge(total_cases_by_area, on='地區', suffixes=('', '_總數'))    
    print(df)
    if sort == "遞增":
        df = df.sort_values(by='案件數量_總數')
        #df = df.groupby('地區', group_keys=False).apply(lambda x: x.sort_values(['案件數量']))
        #df = df.sort_values(by=['地區', '案件數量'])
    elif sort == "遞減":
        df = df.sort_values(by='案件數量_總數', ascending=False)
        #df = df.groupby('地區', group_keys=False).apply(lambda x: x.sort_values(['案件數量'], ascending=False))
        #df = df.sort_values(by=['地區', '案件數量'], ascending=False)

    # 創建長條圖
    figure = px.bar(df, x='地區', y='案件數量', color='年份', barmode="group", title=f"{city}")
    figure.update_layout(title_x=0.5, height=600)
    figure.update_yaxes(tickformat=',')

    # 計算總人數
    result_data = result_data.reset_index()
    result_data.columns = ['年份', '案件數量']
    total_people = result_data["案件數量"].sum()

    return figure, total_people, df

# 製作區域深淺圖
def Create_ggplot(city, df):
    # 載入鄉鎮地區界線數值
    df_dist = gpd.read_file('./mapdata/TOWN_MOI_1120825.shp', encoding='utf-8')
    dist_3 = df_dist.loc[df_dist['COUNTYNAME'].isin([city])]
    #dist_3 = dist_3.to_crs(epsg=3826)
    dist_3.centroid
    dist_3_name = pd.DataFrame({
        'dist': dist_3['TOWNNAME'],
        'long': dist_3.centroid.x,
        'lat': dist_3.centroid.y
    })

    # plot =  ggplot()+\
    #         geom_map(dist_3, fill=None)+\
    #         geom_point(data=dist_3_name,
    #             mapping=aes(x='long', y='lat'), size=1, color='red')+\
    #         geom_label(data=dist_3_name,
    #             mapping=aes(x='long', y='lat', label='dist'),
    #             nudge_y=0.015, family='Arial Unicode MS', boxstyle='round4', color='green',
    #             fill='#ddf52c', alpha=0.8, size=8)+\
    #         theme(figure_size=(3.65, 3.13))
    #ggplot()+geom_map(df_dist, fill=None)+coord_cartesian(xlim=(118, 122.5), ylim=(21.5, 25.5))

    # 資料處理, 獲得繪製深淺圖的資料
    city_pd = pd.DataFrame()
    city_pd["縣市"] = [city for i in range(len(six_capitals_area[city]))]
    city_pd["地區"] = six_capitals_area[city]
    city_grouped_pd = df.groupby('地區')['案件數量'].sum().reset_index()
    city_merge_pd = city_pd.merge(city_grouped_pd, on="地區")
    geo_merge_pd = pd.merge(dist_3[['COUNTYNAME','TOWNNAME','geometry']],
                            city_merge_pd[['縣市','地區','案件數量']],
                            left_on=['COUNTYNAME','TOWNNAME'],
                            right_on=['縣市','地區']).drop(['COUNTYNAME','TOWNNAME'],axis=1)
    
    # 排除旗津區, 因繪製的數值有誤, 會導致圖片失真
    geo_merge_pd = geo_merge_pd[geo_merge_pd['地區'] != '旗津區']

    # 繪製深淺圖
    plot =  ggplot()+\
            geom_map(geo_merge_pd,aes(fill= '案件數量'))+\
            theme(text=element_text(family='Arial Unicode MS', size = 25))+\
            theme(figure_size=(10,8))

    return plot

with output:
    homepage()
display(output)

Output()